In [1]:
import os
os.environ["HOME"] = "/mnt/nas/shuvranshu"

os.environ["HF_HOME"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["HF_DATASETS_CACHE"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["XDG_CACHE_HOME"] = "/mnt/nas/shuvranshu/huggingface_cache"
os.environ["HF_DATASETS_CACHE"] = "/mnt/nas/shuvranshu/huggingface_cache"

os.makedirs("/mnt/nas/shuvranshu/huggingface_cache", exist_ok=True)



In [2]:
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
#hf token 
load_dotenv()  
hf_token = os.getenv("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

/mnt/nas/shuvranshu/.conda/envs/ragenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/nas/shuvranshu/.conda/envs/ragenv/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
llm = HuggingFacePipeline.from_model_id(
    # model_id="/mnt/nas/shuvranshu/huggingface_cache/models--meta-llama--Llama-3.1-8B/snapshots/d04e592bb4f6aa9cfee91e2e20afa771667e1d4b", 
    model_id="meta-llama/Llama-3.1-8B",
    # model_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    model_kwargs={"temperature": 0.1},
    device=2
)


embeddings = HuggingFaceEmbeddings(model_name="/mnt/nas/shuvranshu/huggingface_cache/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/c9745ed1d9f207416be6d2e6f8de32d1f16199bf",model_kwargs={"device": "cuda:2"})

vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 40})


prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Use the following context to answer the question.\nContext: {context}\nQuestion: {question}\nAnswer:"
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


def predict_fn(question: str) -> str:
    return qa.run(question)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.39it/s]
Device set to use cuda:2
/tmp/ipykernel_2390915/864760465.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="/mnt/nas/shuvranshu/huggingface_cache/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/c9745ed1d9f207416be6d2e6f8de32d1f16199bf",model_kwargs={"device": "cuda:2"})
No sentence-transformers model found with name /mnt/nas/shuvranshu/huggingface_cache/models--sentence-transformers--all-Min

In [4]:
from datasets import load_dataset,Dataset
dataset=load_dataset("squad_v2",split="validation[:100]")


In [ ]:
# print(len(dataset[5]["answers"]["text"]))
print(dataset[1]["question"])
print(dataset[1]["context"])
print(dataset[1]["answers"])

When were the Normans in Normandy?
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
{'text': ['10th and 11th centuries', 'in the 10th and 11th centuries', '10th and 11th centuries', '10th and 11th centuries'], 'answer_start': [94, 87, 94, 94]}


In [5]:
questions=[]
ground_truths=[]
q=0
for row in dataset:
    questions.append(row["question"])
    vectorstore.add_texts(row["context"])
    if(len(row["answers"]["text"])):
        ground_truths.append(row["answers"]["text"][0])
    else:
        ground_truths.append("")
    
    
    print(f"question:{q} completed")
    q+=1


question:0 completed
question:1 completed
question:2 completed
question:3 completed
question:4 completed
question:5 completed
question:6 completed
question:7 completed
question:8 completed
question:9 completed
question:10 completed
question:11 completed
question:12 completed
question:13 completed
question:14 completed
question:15 completed
question:16 completed
question:17 completed
question:18 completed
question:19 completed
question:20 completed
question:21 completed
question:22 completed
question:23 completed
question:24 completed
question:25 completed
question:26 completed
question:27 completed
question:28 completed
question:29 completed
question:30 completed
question:31 completed
question:32 completed
question:33 completed
question:34 completed
question:35 completed
question:36 completed
question:37 completed
question:38 completed
question:39 completed
question:40 completed
question:41 completed
question:42 completed
question:43 completed
question:44 completed
question:45 complete

In [6]:

rag_answers=[]
retrieved_contexts=[]
q=0
for question in questions:
    docs=retriever.get_relevant_documents(question)
    context = [d.page_content for d in docs] 
    print(f"question:{q}, no of contexts retrieved:{len(context)}")
    q+=1
    retrieved_contexts.append(context)
    result=qa.run(question)
    rag_answers.append(result)


/tmp/ipykernel_2390915/175219143.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs=retriever.get_relevant_documents(question)
/tmp/ipykernel_2390915/175219143.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa.run(question)


question:0, no of contexts retrieved:40
question:1, no of contexts retrieved:40
question:2, no of contexts retrieved:40
question:3, no of contexts retrieved:40
question:4, no of contexts retrieved:40
question:5, no of contexts retrieved:40
question:6, no of contexts retrieved:40
question:7, no of contexts retrieved:40
question:8, no of contexts retrieved:40
question:9, no of contexts retrieved:40


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


question:10, no of contexts retrieved:40
question:11, no of contexts retrieved:40
question:12, no of contexts retrieved:40
question:13, no of contexts retrieved:40
question:14, no of contexts retrieved:40
question:15, no of contexts retrieved:40
question:16, no of contexts retrieved:40
question:17, no of contexts retrieved:40
question:18, no of contexts retrieved:40
question:19, no of contexts retrieved:40
question:20, no of contexts retrieved:40
question:21, no of contexts retrieved:40
question:22, no of contexts retrieved:40
question:23, no of contexts retrieved:40
question:24, no of contexts retrieved:40
question:25, no of contexts retrieved:40
question:26, no of contexts retrieved:40
question:27, no of contexts retrieved:40
question:28, no of contexts retrieved:40
question:29, no of contexts retrieved:40
question:30, no of contexts retrieved:40
question:31, no of contexts retrieved:40
question:32, no of contexts retrieved:40
question:33, no of contexts retrieved:40
question:34, no 

In [7]:
import pandas as pd

data = pd.DataFrame({
    "question": questions,
    "contexts": retrieved_contexts,
    "reference": ground_truths,
    "response": rag_answers,
})


In [ ]:
# #ragas evaluation
# from langchain_community.llms import HuggingFacePipeline
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from datasets import Dataset


# model_name="meta-llama/Llama-3.2-3B-Instruct"


# tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/mnt/nas/shuvranshu/huggingface_cache")
# ragas_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     cache_dir="/mnt/nas/shuvranshu/huggingface_cache",
#     # device_map="auto",            
#     offload_folder="/mnt/nas/shuvranshu/offload",  
#     dtype=torch.float16,
#     device_map={"": 0} #force gpu usage
# )

# pipe = pipeline(
#     task="text-generation",
#     model=ragas_model,
#     tokenizer=tokenizer,
#     max_new_tokens=128,
#     temperature=0.1,
#     device_map={"": 2}
# )

# ragas_llm = HuggingFacePipeline(pipeline=pipe)


# ragas_dataset = Dataset.from_pandas(data)
# ragas_dataset_small = ragas_dataset.select(range(2))


# # from ragas import evaluate
# # from ragas.metrics import answer_correctness, faithfulness, context_precision
# # from ragas.evaluation import RunConfig


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.16s/it]
Device set to use cuda:0
/tmp/ipykernel_2353995/127511802.py:30: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  ragas_llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:

# config = RunConfig(
#     timeout=7200,      # 2 hour per job
#     max_workers=1      # run serially
# )

# result = evaluate(
#     ragas_dataset_small,
#     metrics=[answer_correctness, faithfulness, context_precision],
#     llm=ragas_llm,
#     embeddings=embeddings,
#     # return_executor=True
# )

# # for job in result.jobs:
# #     job_input, job_output, job_error, job_traceback = job
# #     print("Input:", job_input)
# #     print("Raw model output:", job_output)
# #     # The parsing error is now in job_error
# #     print("Parsing error (if any):", job_error)
# #     print("Full Traceback:", job_traceback)
# #     print("----")
# print(result)

In [8]:
#hallucination score
import re

def simple_hallucination_score(answer: str, context: str) -> float:
    context_words = set(re.findall(r"\w+", str(context).lower()))
    answer_words = re.findall(r"\w+", str(answer).lower())

    if not answer_words:
        return 0.0

    hallucinated_words = [w for w in answer_words if w not in context_words]
    score = len(hallucinated_words) / len(answer_words)
    return score
# from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('all-MiniLM-L6-v2')
# def semantic_hallucination_score(answer: str, context: str, threshold: float = 0.7) -> float:
#     """
#     Computes a hallucination score using semantic similarity.
    
#     Parameters:
#     - answer: generated answer string
#     - context: reference context string
#     - threshold: similarity threshold to consider a word/phrase supported
    
#     Returns:
#     - score: fraction of unsupported content in the answer
#     """
#     if not answer.strip():
#         return 0.0

#     # Split the answer into phrases or sentences
#     answer_sentences = [s.strip() for s in answer.split('.') if s.strip()]
#     context_embedding = model.encode(context, convert_to_tensor=True)
    
#     hallucinated_count = 0
#     for sent in answer_sentences:
#         sent_embedding = model.encode(sent, convert_to_tensor=True)
#         similarity = util.cos_sim(sent_embedding, context_embedding).item()
#         if similarity < threshold:
#             hallucinated_count += 1
    
#     score = hallucinated_count / len(answer_sentences)
#     return score

hallucination_scores=[]
ragas_dataset = Dataset.from_pandas(data)
for entry in ragas_dataset:
    generated_answer = entry["response"]
    context = " ".join(entry["contexts"])  # assuming contexts is a list of strings
    # context=entry["contexts"]
    score = simple_hallucination_score(generated_answer, context)
    hallucination_scores.append(score)

avg_score = sum(hallucination_scores) / len(hallucination_scores)
print(f"\nAverage hallucination score for dataset: {avg_score:.2f}")


Average hallucination score for dataset: 0.44
